In [1]:
import numpy as np
import pandas as pd
import librosa
import whisperx
import torch
import torchaudio
import torchaudio.functional as F
from sklearn.model_selection import train_test_split
import os
from scipy.signal import fftconvolve
import random
import h5py

SAMPLE_RATE = 16000

/work/dpandya/miniconda3/envs/_whisperx_/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/work/dpandya/miniconda3/envs/_whisperx_/lib/python3.10/site-packages/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


In [2]:
device = "cuda"
compute_type = "float16"

model = whisperx.load_model("medium", download_root="models/", device=device, compute_type=compute_type, language='de')

Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.4.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../../home/dpandya/.cache/torch/whisperx-vad-segmentation.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.4.1+cu121. Bad things might happen unless you revert torch to 1.x.


In [3]:
def peak_normalize(audio, target=1.0):
    return audio * (target / np.max(np.abs(audio)))

In [3]:
def add_echo(aud_fname, rir_fname):
    audio, sr = librosa.load(aud_fname, sr=SAMPLE_RATE)
    rir, sr = librosa.load(rir_fname, sr=SAMPLE_RATE)

    augmented = fftconvolve(audio, rir)
    #return peak_normalize(augmented)
    return augmented

In [4]:
libri_dir = '../../LibriVox_Kaggle'
rir_dir = '../audioData/Office_RIR'
rir_list = os.listdir(rir_dir)

df = pd.read_csv('LibriVox_Kaggle_org.csv')

In [5]:
train, test = train_test_split(df, random_state=42)

In [6]:
rir_df = pd.read_csv('whisperx_large_v2/LibriVox_Kaggle_officeRIR_whisperx.csv')
rir_ = rir_df['rir_file']

In [7]:
c = 0
with h5py.File('augmented_arrays.h5', 'w') as f:
    for i, row in test.iterrows():
        aud_file = os.path.join(libri_dir, row['file'])
        rir_file = rir_[c]
        c+=1
        augmented = add_echo(aud_fname=aud_file, rir_fname=rir_file)

        # Create a group for the aud_file
        aud_group = f.create_group(aud_file)

        # Create a dataset for the augmented numpy array
        aud_group.create_dataset('augmented', data=augmented)

In [28]:
output_ = []

for i,row in test.iterrows():
    fname = os.path.join(libri_dir, row['file'])
    aud = whisperx.load_audio(fname)
    results = model.transcribe(aud, language='de')
    for segment in results['segments']:
        output_.append(segment['text'])

In [12]:
output = []

with h5py.File('augmented_arrays.h5', 'r') as f:
    for i, row in test.iterrows():
        fname = os.path.join(libri_dir, row['file'])
        if fname in f:
            augmented = f[fname]['augmented'][:]
            results = model.transcribe(augmented, language='de')
            for segment in results['segments']:
                output.append(segment['text'])


In [15]:
import unicodedata
import re

def normalize_string(s):
    # Convert to lower case
    s = s.lower()
    
    # Remove punctuation
    s = re.sub(r'[^\w\s]', '', s)
    
    # Additional replacements
    s = s.replace('ß', 'ss')
    s = s.replace('ö', 'oe')
    s = s.replace('ä', 'ae')
    s = s.replace('ü', 'ue')
    
    # Convert umlauts and special characters to normal English characters
    s = unicodedata.normalize('NFD', s).encode('ascii', 'ignore').decode('utf-8')
    
    return s

In [30]:
test.columns

Index(['file', 'orgText', 'rir_medium_whisperxOut', 'medium_whisperxOut_wer'], dtype='object')

In [34]:
import jiwer

#output
test['medium_whisperxOut'] = output_

In [35]:
#test['medium_WER'] = jiwer.wer(test['orgText'].tolist(), test['medium_whisperxOut'].tolist())

test['medium_whisperxOut'] = test['medium_whisperxOut'].apply(lambda x: normalize_string(x))
#test['orgText'] = test['orgText'].apply(lambda x: normalize_string(x))

test['medium_whisperxOut_wer'] = test.apply(lambda x: jiwer.wer(x['orgText'], x['medium_whisperxOut']), axis=1)

In [19]:
test=test.drop(columns=['WER'])

In [36]:
test['medium_whisperxOut_wer'].mean()

0.07989959618800023

In [38]:
test = test[['file', 'orgText', 'medium_whisperxOut', 'rir_medium_whisperxOut', 'medium_whisperxOut_wer', 'rir_medium_whisperxOut_wer']]

In [32]:
# Rename columns
test.rename(columns={'medium_whisperxOut': 'rir_medium_whisperxOut', 
                     'medium_whisperxOut_wer': 'rir_medium_whisperxOut_wer'}, inplace=True)

In [39]:
test.to_csv('whisperx_medium/LibriVox_Kaggle_officeRIR_medium_whisperx.csv', index=False)

In [26]:
test.drop(columns=['whisperxOut'], inplace=True)

In [27]:
test

,file,orgText,rir_medium_whisperxOut,medium_whisperxOut_wer
625,achtgesichterambiwasse/achtgesichterambiwasse_...,koennt ihr durch einen zufall drueben in karas...,koennt ihr durch einen zufall drueben in kara...,0.000000
2796,meisterfloh/meisterfloh_0169.wav,peregrinus wollte von der wirklichen welt nich...,peragrinus wollte von der wirklichen welt nic...,0.100000
101,achtgesichterambiwasse/achtgesichterambiwasse_...,und hanake wusste als guterzogene japanerin da...,und hanake wusste als gut erzogene japanerin ...,0.166667
4766,meisterfloh/meisterfloh_2139.wav,nicht genug dass der taugenichts ueberall wo e...,nicht genug dass der tauge nichts ueberall wo...,0.117647
2018,achtgesichterambiwasse/achtgesichterambiwasse_...,entdeckte sie ploetzlich einen mann der ihrem ...,entdeckte sie ploetzlich einen mann der ihrem...,0.000000
...,...,...,...,...
3524,meisterfloh/meisterfloh_0897.wav,fuehrt weil er in allen dingen des lebens zur ...,fuehrt weil er in allen dingen des lebens zur...,0.000000
6971,serapionsbruederauswahl/serapionsbruederauswah...,dieses juenglings glueckliche braut vor funfzi...,dieses juenglings glueckliche braut vor fuenf...,0.142857
3080,meisterfloh/meisterfloh_0453.wav,es befand sich zu der zeit ein mann in frankfu...,es befand sich zu der zeit ein mann in frankf...,0.250000
3187,meisterfloh/meisterfloh_0560.wav,und fiel bald in tiefen schlaf gerade der fein...,und fiel bald in tiefen schlaf gerade der fei...,0.000000


In [7]:
output = []
# Open the HDF5 file
with h5py.File('augmented_arrays.h5', 'r') as f:
    # Iterate over the datasets in the file
    for i, row in test.iterrows():
        if row['file'] in f:
            augmented = f[row['file']][rir_file]['augmented'][:]
            results = model.transcribe(augmented, language='de')
            for segment in results['segments']:
                output.append(segment['text'])

In [8]:
output

[]

In [22]:
test['medium_whisperxOut'] = output
test['rir_file'] = rir_

In [23]:
test.head(5)

,file,orgText,whisperxOut,WER,medium_whisperxOut,rir_file
625,achtgesichterambiwasse/achtgesichterambiwasse_...,könnt ihr durch einen zufall drüben in karasak...,Könnt ihr durch einen Zufall drüben in Karasa...,NaN,Das Mädchen war leise aufgestanden und hatte ...,../audioData/Office_RIR/h110_Office_MeetingRoo...
2796,meisterfloh/meisterfloh_0169.wav,peregrinus wollte von der wirklichen welt nich...,Peregrinus wollte von der wirklichen Welt nic...,NaN,Das Mädchen war leise aufgestanden und hatte ...,NaN
101,achtgesichterambiwasse/achtgesichterambiwasse_...,und hanake wusste als guterzogene japanerin da...,"und Hanake wußte als gut erzogene Japanerin, ...",NaN,Das Mädchen war leise aufgestanden und hatte ...,../audioData/Office_RIR/h173_Offixe_1txts.wav
4766,meisterfloh/meisterfloh_2139.wav,nicht genug dass der taugenichts überall wo er...,"Nicht genug, dass der Taugenichts überall, wo...",NaN,Das Mädchen war leise aufgestanden und hatte ...,NaN
2018,achtgesichterambiwasse/achtgesichterambiwasse_...,entdeckte sie plötzlich einen mann der ihrem s...,"Entdeckte sie plötzlich einen Mann, der ihrem...",NaN,Das Mädchen war leise aufgestanden und hatte ...,NaN


In [36]:
out = []
#rir_files = []

for i, row in test.iterrows():
    aud_file = os.path.join(libri_dir, row['file'])
    rir_file = rir_.iloc[i]

    augmented = add_echo(aud_fname=aud_file, rir_fname=rir_file)
    results = model.transcribe(augmented, language='de')

    for segment in results['segments']:
        out.append(segment['text'])

In [37]:
test['rir_whisperxOut'] = out
test['rir_file'] = rir_files


In [39]:
test = test[['file', 'rir_file', 'orgText', 'whisperxOut', 'rir_whisperxOut', 'WER']]

In [40]:
test.to_csv('LibriVox_Kaggle_officeRIR_whisperx.csv', index=False)